In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
import warnings as wr
wr.filterwarnings('ignore')


In [ ]:
df=pd.read_csv("../input/loan-application-data/df1_loan.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.describe().T.style.bar(subset=["mean"],color="#606ff2").background_gradient(subset=["std"],cmap="PuBu").background_gradient(subset=["50%"],cmap="PuBu")

# Data Preposesing


In [ ]:
df.tail()


* Here we can see there are two unnessary column (unnamed: 0 and Loan_ID) 
* so we remove them

In [ ]:
#df.columns=[column.replace(" ","_") for column in df.columns]
df=df.drop(["Unnamed: 0","Loan_ID"],axis=1)

In [ ]:
#removing doller sign from Total_Income column
df['Total_Income']=df['Total_Income'].str.replace('$','')

In [ ]:
df["Total_Income"]=df["Total_Income"].apply(pd.to_numeric)

In [ ]:
#removing + sign from dependents column
df['Dependents']=df['Dependents'].str.replace('+','')

In [ ]:
df["Dependents"]=df["Dependents"].apply(pd.to_numeric)

## Dealing with null values
Now our goal is to deal with null values and try to understand for each one what can we do: maybe we can replace them or maybe we can just skip them.

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df.isnull(),yticklabels=False,cbar=True,cmap='mako')

In [ ]:
df.isnull().sum()

* Here we can see there are many featuers have NaN values.
* Now we fill all the missing values using pandas fill na 
### Now we print percentage of null values

In [ ]:
total_null = df.isnull().sum().sort_values(ascending=False) #First sum and order all null values for each variable
percentage = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False) #Get the percentage
missing_data = pd.concat([total_null, percentage], axis=1, keys=['Total', 'Percentage'])
missing_data.head()

* Now we can see there are min missing values so we need to fill them
* **Finding numeric column**

In [ ]:
num_col=df._get_numeric_data().columns.tolist()
num_col

## finding cat column

In [ ]:
cat_col=set(df.columns)-set(num_col)
cat_col


## Filling numerical missing values

In [ ]:
for col in num_col:
    df[col].fillna(df[col].mean(),inplace=True)

## Filling categorical missing values

In [ ]:
for col in cat_col:

    df[col].fillna(df[col].mode()[0],inplace=True)

In [ ]:
df.isnull().sum()

## Checking for Duplicate

In [ ]:
df.duplicated().sum()

In [ ]:
plt.figure(figsize=(12,6))
ax=plt.axes()
ax.set_facecolor("green")
for i in cat_col:
    sns.countplot(df[i])

* Here we find zero duplicated values
## Now finding correlation

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),annot=True,cmap="tab20c")
plt.show()

In [ ]:
#ApplicantIncome analysis

sns.distplot(df['ApplicantIncome']);
print("Skewness coeff. is: %f" % df['ApplicantIncome'].skew())
print("Kurtosis coeff. is: %f" % df['ApplicantIncome'].kurt())

In [ ]:
#CoapplicantIncome analysis

sns.distplot(df['CoapplicantIncome']);
print("Skewness coeff. is: %f" % df['CoapplicantIncome'].skew())
print("Kurtosis coeff. is: %f" % df['CoapplicantIncome'].kurt())

In [ ]:
sns.kdeplot(data=df,x='ApplicantIncome',hue="Gender",fill=True,common_norm=False,palette="husl")

In [ ]:
x=df.drop(["Loan_Status"],axis=1)
y=df["Loan_Status"]
x.head()

# Model Buliding

In [ ]:
X=pd.get_dummies(x)
X.columns

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y=lb.fit_transform(y)



In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.30,random_state=10)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(X_train,y_train)

In [ ]:
from sklearn.svm import SVC
svc = SVC()

svc.fit(X_train, y_train)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtf = DecisionTreeClassifier()
dtf.fit(X_train, y_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB
n_b = GaussianNB()
n_b.fit(X_train, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()  
knn.fit(X_train, y_train)

In [ ]:
print(lr.score(X_test, y_test))
print(dtf.score(X_test, y_test))
print(n_b.score(X_test, y_test))
print(knn.score(X_test, y_test))
print(svc.score(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix 
y_predict=lr.predict(X_test)
results = confusion_matrix(y_test, y_predict)
from sklearn.metrics import confusion_matrix 
y_predict=lr.predict(X_test)
results = confusion_matrix(y_test, y_predict) 
print('Confusion Matrix :')
print(results)

print('Confusion Matrix :')
print(results)



In [ ]:
print(lr.predict_proba(X_test[0:5]))

### roc_curve, auc,roc_auc_score

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
fpr,tpr,threshold = roc_curve(y_test,lr.predict_proba(X_test)[:,1])
fpr1,tpr1,threshold1 = roc_curve(y_test,knn.predict_proba(X_test)[:,1])


In [ ]:
fpr

In [ ]:
tpr

In [ ]:
threshold

In [ ]:
roc_auc=roc_auc_score(y_test,lr.predict(X_test))
roc_auc1=roc_auc_score(y_test,knn.predict(X_test))


In [ ]:
roc_auc1

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw,label='ROC curve (area = %0.2f)' % roc_auc)

plt.plot(fpr1, tpr1, color='blue',
         lw=lw,label='ROC curve (area = %0.2f)' % roc_auc1)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.1])
plt.ylim([0.0, 1.1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_predict=lr.predict_proba(X_test)
y_predict

In [ ]:

from sklearn.preprocessing import binarize

y_pred_class=binarize(y_predict,0.60)
#y_pred_class

In [ ]:
y_pred_class1=y_pred_class[:,1]
y_pred_class1

In [ ]:
y_pred=y_pred_class1.astype(int)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix 
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results)



In [ ]:
y_pred_class=binarize(y_predict,0.70)
y_pred_class1=y_pred_class[:,1]
y_pred=y_pred_class1.astype(int)
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

y_pred=rfc.predict(X_test)
print(rfc.score(X_test, y_test))



In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# *Thank YOU*